# EDA insights

## Intro

In this notebook I will focus on questions related with Public Policy that could have been done by Government Institutions:

1. How the Colombian student population is distributed around the country?
2. Which is the average grade per Department by Subject in each test (Saber-11 & Saber-PRO)?
3. 
4. 

_Note: these questions reflect my own ideas, or questions stated in documents shared by the owner of the project, and could be different of questions asked by Colombian public servants._

## Initial step
Import libraries, load and translate dataset.

In [1]:
# Import libraries

# basic libraries
import numpy as np
import pandas as pd
import re
# import ast
pd.set_option('display.max_rows', 500) # to show more rows.

# for translation
import sys
sys.path.append('../')
from Translate_data import translate_data

# to plot
import altair as alt
# alt.renderers.enable('mimetype')
alt.renderers.enable('default')

# root to call the data 
root = "../../../data/"
colombia_topo_url = "https://raw.githubusercontent.com/OmdenaAI/omdena-colombia-career-recommender-system/main/src/data/Colombia_departamentos_municipios_poblacion-topo.json"

In [2]:
# Load Data
df = pd.read_csv(root + 'saber_combined_all_fields.csv')
df = df.drop(columns = 'Unnamed: 0')

In [3]:
# Translate data
df_eng = translate_data(df, 'spanish', 'english')

In [4]:
# Wrangling Department

# - - - correct code - - - 
# Change code from float to int
df_eng['your_dept_code_resides'] = df_eng[['your_dept_code_resides']].apply(np.int64).astype(str)
df_eng['your_dept_code_resides.1'] = df_eng[['your_dept_code_resides.1']].apply(np.int64).astype(str)

# two-digit code for Department (Colombian States)
df_eng['your_dept_code_resides'] = ["0"+i if len(i)==1 else i for i in df_eng['your_dept_code_resides']]
df_eng['your_dept_code_resides.1'] = ["0"+i if len(i)==1 else i for i in df_eng['your_dept_code_resides.1']]


In [5]:
# Wrangling Municipality

# - - - correct name - - - 
import re
municipality_list = list(df_eng['cole_municipality_location'].unique()) + \
                    list(df_eng['your_municipality_resides'].unique()) + \
                    list(df_eng['your_inst_municipality'].unique()) + \
                    list(df_eng['your_municipality_resides.1'].unique())

municipality_list = pd.DataFrame(data=municipality_list, columns=['Name'])#.unique()
municipality_list = list(municipality_list.Name.unique())
municipality_list = [str(i) for i in municipality_list]
municipality_list = sorted(municipality_list)
municipality_values = sorted(municipality_list)

municipality_values = [re.sub(r'BOGOTÁ, D.C.', 'BOGOTÁ D.C.', mun) for mun in municipality_values]
municipality_values = [re.sub(r'CARTAGENA$', 'CARTAGENA DE INDIAS', mun) for mun in municipality_values]
municipality_values = [re.sub(r'CHIQUINQUIRA$', 'CHIQUINQUIRÁ', mun) for mun in municipality_values]
municipality_values = [re.sub(r'CIÉNEGA$', 'CIÉNAGA', mun) for mun in municipality_values]
municipality_values = [re.sub(r'PUERTO ASIS$', 'PUERTO ASÍS', mun) for mun in municipality_values]
municipality_values = [re.sub(r'POPAYAN$', 'POPAYÁN', mun) for mun in municipality_values]
municipality_values = [re.sub(r'FACATATIVÁ$', 'FACATATIVA', mun) for mun in municipality_values]

zip_iterator = zip(municipality_list, municipality_values) # Get pairs of elements
municipality_dict = dict(zip_iterator)  # Convert to dictionary

municipality_columns = ['cole_municipality_location', 'your_municipality_resides',
                        'your_inst_municipality', 'your_municipality_resides.1']

for col in municipality_columns:
    df_eng[col] = df_eng[col].map(municipality_dict)
    
# - - - correct code - - - 
# Change code from float to int
df_eng['your_municipality_code_resides'] = df_eng[['your_municipality_code_resides']].apply(np.int64).astype(str)
df_eng['your_municipality_code_resides.1'] = df_eng[['your_municipality_code_resides.1']].apply(np.int64).astype(str)

# two-digit code for Department (Colombian States)
df_eng['your_municipality_code_resides'] = ["0"+i if len(i)==4 else i for i in df_eng['your_municipality_code_resides']]
df_eng['your_municipality_code_resides.1'] = ["0"+i if len(i)==4 else i for i in df_eng['your_municipality_code_resides.1']]

In [6]:
# df_eng[df_eng['your_dept_code_resides'] == '05'][['your_dept_code_resides', 'your_dept_resides', 'your_municipality_code_resides', 'your_municipality_resides']]

In [7]:
df_eng.tail(3)

,your_type_of_document,your_nationality,your_gender,your_birthdate,your_foreigner,period,your_consecutive,your_marital_status,your_student,your_country_resides,...,score_biology_saber_11,score_chemistry_saber_11,score_physics_saber_11,score_social_science_saber_11,score_philosophy_saber_11,score_english_saber_11,score_english_saber_11_category,optative_field_saber_11,score_optative_saber_11,optative_category_saber_11
212007,CC,COLOMBIA,F,21/10/1971,NO,20183,EK201830110470,Married,STUDENT,COLOMBIA,...,43.0,29.0,46.0,40.0,34.0,38.0,A-,SCORE_INTERDISC_ENVIRONMENT,52.0,NaN
212008,CC,COLOMBIA,M,10/08/1994,NO,20183,EK201830148018,Single,STUDENT,COLOMBIA,...,42.0,46.0,45.0,44.0,33.0,43.0,A-,SCORE_INTERDISC_ENVIRONMENT,52.0,NaN
212009,CC,COLOMBIA,M,01/01/1997,NO,20183,EK201830033403,Single,STUDENT,COLOMBIA,...,57.0,61.0,56.0,69.0,69.0,50.0,A1,SCORE_DEEPEN_MATHEMATICS,6.0,II


In [8]:
# Department of residence (Colombia states)
source = df_eng.groupby('your_dept_resides').count()[['your_consecutive']].reset_index().rename(columns={'your_consecutive':'obs'})
print("\nNumber of Departments (States) in Colombia:", len(source), "\n")

alt.Chart(source).mark_bar().encode(
    alt.X('your_dept_resides', sort='-y', title="Department"),
    alt.Y('obs', title="Observations", scale=alt.Scale(type='log')),
    alt.Color('your_dept_resides', legend=None, sort='-y', 
              scale=alt.Scale(scheme='yellowgreenblue', reverse=True)),
    tooltip=['your_dept_resides', 'obs']
).properties(
    title = "Number of students per Department of Colombia (State)"
)


Number of Departments (States) in Colombia: 34 



alt.Chart(...)

In [9]:
# Municipality
source = df_eng.groupby('your_municipality_resides').count()[['your_consecutive']].reset_index().rename(columns={'your_consecutive':'obs'})

print("\nNumber of Municipalities in Colombia:", len(source), "\n")

alt.Chart(source).mark_bar(opacity=0.9).encode(
    alt.X('your_municipality_resides', sort='-y', title="Municipality"),
    alt.Y('obs', title="Observations", scale=alt.Scale(type='log')),
    alt.Color('your_municipality_resides', legend=None, sort='-y',
             scale=alt.Scale(scheme='yellowgreenblue', reverse=True)),
    tooltip=['your_municipality_resides', 'obs']
).properties(
    width=8500,
    title = "Number of students per Municipality"
)


Number of Municipalities in Colombia: 961 



alt.Chart(...)

#### - - - Distribution by maps - - -

In [83]:
# Plot number of students in Colombia by Department (State)

### DATA ###
# States Polygons
states = alt.topo_feature(colombia_topo_url, 'MGN_AMN_DPTOS')

# Dataframe of the project 'group by' for plotting
source = df_eng.groupby(['your_dept_resides', 'your_dept_code_resides']).\
        count()[['your_consecutive']].reset_index().rename(columns={'your_consecutive':'obs',
                                                                    'your_dept_resides': 'name',
                                                                    'your_dept_code_resides': 'code'})

### PLOT ###
# background
base_depart  = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='red'
).properties(
    width=500,
    height=300
)

# find Department by code
plot_depart_by_code = alt.Chart(states).mark_geoshape(stroke='salmon').encode(
    alt.Color('obs:Q', scale=alt.Scale(type='log'), title="Students"),
#     tooltip=[alt.Tooltip('your_dept_resides:N'), alt.Tooltip('obs:Q')]
).transform_lookup(
    lookup='properties.DPTO_CCDGO',
    from_=alt.LookupData(data=source, key='code', fields=['obs']),
#     tooltip=['lookup:O', 'obs:Q']
).properties(
    width=500,
    height=300
)

# Integrate the plots
plot_depart = (base_depart + plot_depart_by_code)\
                    .properties(title="Number of students in Colombia by Department (State)")\
                    .encode(tooltip=['properties.DPTO_CNMBR:N', 'properties.DPTO_CCDGO:Q', 'obs:Q'])
plot_depart

alt.LayerChart(...)

In [84]:
# Plot number of students in Colombia by Municipality

### DATA ###
# Municipalitys Polygons
municipalities = alt.topo_feature(colombia_topo_url, 'MGN_AMN_MPIOS')

# Dataframe of the project 'group by' for plotting
source = df_eng.groupby(['your_municipality_resides', 'your_municipality_code_resides']).\
        count()[['your_consecutive']].reset_index().rename(columns={'your_consecutive':'obs',
                                                                    'your_municipality_resides': 'name',
                                                                    'your_municipality_code_resides': 'code'})

### PLOT ###
# background
base_municip  = alt.Chart(municipalities).mark_geoshape(
    fill='lightgray',
    stroke='salmon'
).properties(
    width=500,
    height=300
)
 

# find Department by code
plot_municip_by_code = alt.Chart(municipalities).mark_geoshape(opacity=0.8, stroke='salmon').encode(
    alt.Color('obs:Q', scale=alt.Scale(type='log'), title="Students")
).transform_lookup(
    lookup='properties.MPIO_CDPMP',
    from_=alt.LookupData(data=source, key='code', fields=['obs'])
).properties(
    width=500,
    height=300
)

# find Department by Name
plot_municip_by_name = alt.Chart(municipalities).mark_geoshape(opacity=0.8).encode(
    alt.Color('obs:Q', scale=alt.Scale(type='log'), title="Students")
).transform_lookup(
    lookup='properties.MPIO_CNMBR',
    from_=alt.LookupData(data=source, key='name', fields=['obs'])
).properties(
    width=500,
    height=300
)

# Integrate the plots
plot_municip = (base_municip + plot_municip_by_name + plot_municip_by_code)\
                    .properties(title="Number of students in Colombia by Municipality")\
                    .encode(tooltip=['properties.MPIO_CNMBR:N', 'properties.MPIO_CCDGO:Q', 'obs:Q'])
plot_municip

alt.LayerChart(...)

### 2. Which is the average grade per Department by Subject in each test (Saber-11 & Saber-PRO)?
#### - - - Saber-11 - - - 

In [85]:
# Average of Saber-11 scores per Department by Subject

source = pd.DataFrame(columns=['your_dept_resides', 'your_dept_code_resides', 'average', 'subject'])
saber_11_scores = ['score_language_saber_11', 'score_mathematics_saber_11', 'score_biology_saber_11', 
                   'score_chemistry_saber_11', 'score_physics_saber_11', 'score_social_science_saber_11', 
                   'score_philosophy_saber_11', 'score_english_saber_11', 'score_optative_saber_11']
saber_11_subjects = ['biology', 'chemistry', 'english', 'language', 'mathematics', 'optative', 'philosophy', 'physics', 'social_science']

for i in saber_11_scores:
    aux = df_eng.groupby(['your_dept_resides', 'your_dept_code_resides']).mean()[[i]].reset_index().rename(columns={i:'average'})
    aux.sort_values(by=['average'], ascending=False, inplace=True)
    aux = pd.concat([aux.head(20), aux.tail(20)])
    aux['subject'] = re.sub(r'_saber_11', '', re.sub(r'score_', '', i)) # Cleans the subject text
    source = pd.concat([source, aux])

source = pd.pivot_table(source, values='average',
                        index=['your_dept_resides', 'your_dept_code_resides'],
                        columns=['subject']).reset_index().rename(columns={'your_dept_resides': 'name',
                                'your_dept_code_resides': 'code'})

### PLOT ###
# find Department by code
plot_saber11_by_code = base_depart.mark_geoshape(stroke='salmon').encode(
#     alt.Color('average:Q', title="Score"), #scale=alt.Scale(type='log'),
    alt.Color(alt.repeat('row'), type='quantitative'),
    tooltip=['properties.DPTO_CNMBR:N', 'properties.DPTO_CCDGO:Q']
).transform_lookup(
    lookup='properties.DPTO_CCDGO',
    from_=alt.LookupData(data=source, key='code', fields=saber_11_subjects)
).properties(
    width=500,
    height=300
).repeat(
    row=saber_11_subjects
).resolve_scale(
    color='independent'
)

plot_saber11_by_code.properties(title="Average of Saber-11 scores per Department by Subject")

alt.RepeatChart(...)

#### - - - Saber-PRO - - -

In [86]:
# Average of Saber-PRO scores per Department by Subject

source = pd.DataFrame(columns=['your_dept_resides.1', 'your_dept_code_resides.1', 'average', 'subject'])
saber_PRO_scores = ['score_math_saber_pro', 'score_language_saber_pro', 'score_social_science_saber_pro',
                    'score_english_saber_pro', 'score_writing_saber_pro']
saber_PRO_subjects = ['english', 'language', 'math', 'social_science', 'writing']

for i in saber_PRO_scores:
    aux = df_eng.groupby(['your_dept_resides.1', 'your_dept_code_resides.1']).mean()[[i]].reset_index().rename(columns={i:'average'})
    aux.sort_values(by=['average'], ascending=False, inplace=True)
    aux = pd.concat([aux.head(20), aux.tail(20)])
    aux['subject'] = re.sub(r'_saber_pro', '', re.sub(r'score_', '', i)) # Cleans the subject text
    source = pd.concat([source, aux])

source = pd.pivot_table(source, values='average',
                        index=['your_dept_resides.1', 'your_dept_code_resides.1'],
                        columns=['subject']).reset_index().rename(columns={'your_dept_resides.1': 'name',
                                'your_dept_code_resides.1': 'code'})

### PLOT ###
# find Department by code
plot_saberPRO_by_code = base_depart.mark_geoshape(stroke='salmon').encode(
#     alt.Color('average:Q', title="Score"), #scale=alt.Scale(type='log'),
    alt.Color(alt.repeat('row'), type='quantitative'),
    tooltip=['properties.DPTO_CNMBR:N', 'properties.DPTO_CCDGO:Q']
).transform_lookup(
    lookup='properties.DPTO_CCDGO',
    from_=alt.LookupData(data=source, key='code', fields=saber_PRO_subjects)
).properties(
    width=500,
    height=300
).repeat(
    row=saber_PRO_subjects
).resolve_scale(
    color='independent'
)

plot_saberPRO_by_code.properties(title="Average of Saber-PRO scores per Department by Subject")

alt.RepeatChart(...)

## Dominant Cluster

In [14]:
# Separating the scores into respective lists

marks = [
 'score_language_saber_11',
 'score_mathematics_saber_11',
 'score_biology_saber_11',
 'score_chemistry_saber_11',
 'score_physics_saber_11',
 'score_social_science_saber_11',
 'score_philosophy_saber_11',
 'score_english_saber_11',
 'score_optative_saber_11', 
 'score_math_saber_pro',
 'score_language_saber_pro',
 'score_social_science_saber_pro',
 'score_english_saber_pro',
 'score_writing_saber_pro'
 ]

saber_marks = [ 
 'score_language_saber_11',
 'score_mathematics_saber_11',
 'score_biology_saber_11',
 'score_chemistry_saber_11',
 'score_physics_saber_11',
 'score_social_science_saber_11',
 'score_philosophy_saber_11',
 'score_english_saber_11',
 'score_optative_saber_11',
 ]

saber_pro_marks = [
 'score_math_saber_pro',
 'score_language_saber_pro',
 'score_social_science_saber_pro',
 'score_english_saber_pro',
 'score_writing_saber_pro'
 ]

# Adding column of total saber and saber pro scores   <<< WHY TO ADD?? NAD NOT TO AVERAGE FOR EXAMPLE?

df_eng['total_saber_score'] = df_eng[saber_marks].sum(axis=1)
df_eng['total_saber_pro_score'] = df_eng[saber_pro_marks].sum(axis=1)

total_marks = ['total_saber_score', 'total_saber_pro_score']

In [15]:
# divide the `fami_monthly_family_income` variable in saber 11 and saber pro
df_eng['fami_monthly_family_income_11'] = df_eng.fami_monthly_family_income.iloc[:,0]
df_eng['fami_monthly_family_income_pro'] = df_eng.fami_monthly_family_income.iloc[:,1]
#df.drop('fami_monthly_family_income')

In [16]:
%%time

# Run this to generate clusters

# Import module for k-protoype cluster
# run fro terminal: pip install kmodes
from kmodes.kprototypes import KPrototypes

columns = ['score_language_saber_11',
 'score_mathematics_saber_11',
 'score_biology_saber_11',
 'score_chemistry_saber_11',
 'score_physics_saber_11',
 'score_social_science_saber_11',
 'score_philosophy_saber_11',
 'score_english_saber_11',
 'score_optative_saber_11',
 'score_math_saber_pro',
 'score_language_saber_pro',
 'score_social_science_saber_pro',
 'score_english_saber_pro',
 'score_writing_saber_pro',
 'total_saber_score',
 'total_saber_pro_score',
 'your_course_teacher_ies',
 'your_course_external_support_ies',
 'your_course_external_ies',
 'fami_housing_stratum',
 'fami_father_education',
 'fami_mother_education',
 'fami_father_occupation',
 'fami_mother_occupation',
#  'fami_monthly_family_income',
 'fami_monthly_family_income_11',
 'fami_monthly_family_income_pro',
 'fami_computer',
 'fami_internet',
 'fami_automobile',
 'your_marital_status',
 'fami_econ_dependents',
 'your_high_school_degree',
 'your_undergraduate_core',
 'your_municipality_resides',
 'your_municipality_code_resides',
 'your_prgm_municipality',
 'your_dept_resides',
 'your_dept_code_resides',
 'your_prgm_department']

cluster_df = df_eng[columns]

cluster_df["your_course_teacher_ies"].fillna("Unknown", inplace=True)
cluster_df["your_course_external_support_ies"].fillna("Unknown", inplace=True)
cluster_df["your_course_external_ies"].fillna("Unknown", inplace=True)
cluster_df['fami_housing_stratum'].fillna("Unknown", inplace=True)
cluster_df["fami_father_education"].fillna("Unknown", inplace=True)
cluster_df["fami_mother_education"].fillna("Unknown", inplace=True)
cluster_df["fami_father_occupation"].fillna("Unknown", inplace=True)
cluster_df["fami_mother_occupation"].fillna("Unknown", inplace=True)
cluster_df['fami_internet'].fillna("No", inplace=True)
cluster_df["fami_computer"].fillna("No", inplace=True)
cluster_df['fami_automobile'].fillna("Unknown", inplace=True)
cluster_df["your_marital_status"].fillna("Unknown", inplace=True)
cluster_df['fami_econ_dependents'].fillna("Unknown", inplace=True)
cluster_df["your_municipality_resides"].fillna("BOGOTÁ D.C.", inplace=True)
cluster_df["your_dept_resides"].fillna("BOGOTA", inplace=True)
cluster_df['your_high_school_degree'].fillna("Academic Bachelor", inplace=True)
cluster_df["your_undergraduate_core"].fillna("UNCLASSIFIED", inplace=True)
cluster_df["score_writing_saber_pro"].fillna(cluster_df['score_writing_saber_pro'].median(), inplace=True)
# cluster_df["fami_monthly_family_income"].fillna("Unknown", inplace=True)
cluster_df["fami_monthly_family_income_11"].fillna("Unknown", inplace=True)
cluster_df["fami_monthly_family_income_pro"].fillna("Unknown", inplace=True)

catColumnsPos = [cluster_df.columns.get_loc(col) for col in list(cluster_df.select_dtypes('object').columns)]
print('Categorical columns           : {}'.format(list(cluster_df.select_dtypes('object').columns)))
print('Categorical columns position  : {}'.format(catColumnsPos))

dfMatrix = cluster_df.to_numpy()
dfMatrix

# Fit the cluster
kprototype = KPrototypes(n_jobs = -1, n_clusters = 3, init = 'Huang', random_state = 0)
kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)

# Add the cluster to the dataframe
cluster_df['Cluster Labels'] = kprototype.labels_
cluster_df['Segment'] = cluster_df['Cluster Labels'].map({0:'upper_class', 1:'middle_class', 2:'lower_class'})

# Save the data with cluster
cluster_df.to_csv(root + "Saber_Cluster_All.csv") 

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Categorical columns           : ['your_course_teacher_ies', 'your_course_external_support_ies', 'your_course_external_ies', 'fami_housing_stratum', 'fami_father_education', 'fami_mother_education', 'fami_father_occupation', 'fami_mother_occupation', 'fami_monthly_family_income_11', 'fami_monthly_family_income_pro', 'fami_computer', 'fami_internet', 'fami_automobile', 'your_marital_status', 'fami_econ_dependents', 'your_high_school_degree', 'your_undergraduate_core', 'your_municipality_resides', 'your_municipality_code_resides', 'your_prgm_municipality', 'your_dept_resides', 'your_dept_code_resides', 'your_prgm_department']
Categorical columns position  : [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [102]:
### Excercise to identify the Class clusters:
cluster_df.columns

xxx = cluster_df[(cluster_df.fami_father_education == 'Graduate Degree') & 
                 (cluster_df.fami_mother_education == 'Graduate Degree') &
                 (cluster_df.fami_housing_stratum == 'Stratum 6')][['fami_father_education',
           'fami_mother_education', 'fami_housing_stratum', 'fami_internet', 
           'your_undergraduate_core', 'Cluster Labels', 'Segment']]

xxx.groupby(['Cluster Labels', 'Segment']).count()

# Upper class segment hev dominant parents that have 'Graduate education' and 
# belongs to 'Stratum 6'. So, Cluster label '0' corresponds to the 'upper class'.
# Reference: https://docs.google.com/document/d/10edCaRBIaH1sDctkABfdUs482c3PJW8Q4QoK286IaeU/edit?usp=sharing

,,fami_father_education,fami_mother_education,fami_housing_stratum,fami_internet,your_undergraduate_core
Cluster Labels,Segment,,,,,
0,upper_class,1053,1053,1053,1053,1053
1,middle_class,1,1,1,1,1
2,lower_class,135,135,135,135,135


In [103]:
# Plot clusters by Department (State)

# ### DATA ###
aux_cluster = cluster_df[['your_dept_resides', 'your_dept_code_resides','Segment']]
source = pd.pivot_table(aux_cluster, index=['your_dept_resides', 'your_dept_code_resides'], 
                         columns=['Segment'], aggfunc=lambda x: len(x)).reset_index().fillna(0)

# find the dominant cluster
source['dominant_cluster'] = source[['lower_class', 'middle_class', 'upper_class']].idxmax(axis=1)
cluster_dpto = source.copy()

### PLOT ###
domain_ = ['lower_class', 'middle_class', 'upper_class']
range_ = ['lightgreen', 'orange', 'purple']

# clusering by Department
plot_depart_by_code = alt.Chart(states).mark_geoshape(stroke='white').encode(
    alt.Color('dominant_cluster:N',  title="Dominant Cluster",
             scale=alt.Scale(domain=domain_, range=range_)),
).transform_lookup(
    lookup='properties.DPTO_CCDGO',
    from_=alt.LookupData(data=source, key='your_dept_code_resides', fields=['dominant_cluster', 'lower_class', 'middle_class', 'upper_class']),
).properties(
    width=500,
    height=300
)

plot = (base_depart + plot_depart_by_code)\
                    .properties(title="Dominant cluster in Colombia by Department")\
                    .encode(tooltip=['properties.DPTO_CNMBR:N', 'properties.DPTO_CCDGO:Q', 'dominant_cluster:N',
                                     'lower_class:Q', 'middle_class:Q', 'upper_class:Q'])
plot

alt.LayerChart(...)

In [104]:
print('\ndepartments (states)', len(source), '\n')
print(3+5+26)
# source[source['dominant_cluster'] == "First"]

xxx = source.groupby(['dominant_cluster']).agg(['count'])[['your_dept_resides']]
xxx
source[source.dominant_cluster == 'upper_class']


departments (states) 35 

34


Segment,your_dept_resides,your_dept_code_resides,lower_class,middle_class,upper_class,dominant_cluster
1,ANTIOQUIA,05,8383.0,6161.0,9722.0,upper_class
4,BOGOTA,-9223372036854775808,153.0,53.0,159.0,upper_class
5,BOGOTA,11,20169.0,13478.0,29882.0,upper_class
16,EXTRANJERO,99999,5.0,0.0,14.0,upper_class


In [105]:
xxx / (3+5+26)

,your_dept_resides
,count
dominant_cluster,
lower_class,0.764706
middle_class,0.147059
upper_class,0.117647


To complement the cluster analysis, we map the dominant clusters per each department which are equivalent to Colombian’s States. In the following map, we observe that the larger number of departments belong to cluster 3, in fact 26 out of 34 entities (76.5%). Only 5 states belong to the Second cluster (14.7%), and 3 entities -considering the Capital District Bogota- to the First cluster (8.8%).

In [106]:
# Plot clusters by Municipality

# ### DATA ###
aux_cluster = cluster_df[['your_municipality_resides', 'your_municipality_code_resides', 'Segment']]
source = pd.pivot_table(aux_cluster, index=['your_municipality_resides', 'your_municipality_code_resides'], 
                         columns=['Segment'], aggfunc=lambda x: len(x)).reset_index().fillna(0)

# find the dominant cluster
source['dominant_cluster'] = source[['lower_class', 'middle_class', 'upper_class']].idxmax(axis=1)
cluster_mpio = source.copy()

### PLOT ###
domain_ = ['lower_class', 'middle_class', 'upper_class']
range_ = ['lightgreen', 'orange', 'purple']

# clusering by Municipality
plot_municip = alt.Chart(municipalities).mark_geoshape(stroke='white').encode(
    alt.Color('dominant_cluster:N',  title="Dominant Cluster", sort=['lower_class', 'middle_class', 'upper_class'],
             scale=alt.Scale(domain=domain_, range=range_)),
).transform_lookup(
    lookup='properties.MPIO_CNMBR',
    from_=alt.LookupData(data=source, key='your_municipality_resides', fields=['dominant_cluster', 'lower_class', 'middle_class', 'upper_class']),
).properties(
    width=500,
    height=300
)

plot = (base_municip + plot_municip)\
                    .properties(title="Dominant cluster in Colombia by Municipality")\
                    .encode(tooltip=['properties.MPIO_CNMBR:N', 'properties.MPIO_CCDGO:Q', 'dominant_cluster:N',
                                     'lower_class:Q', 'middle_class:Q', 'upper_class:Q'])
plot

alt.LayerChart(...)

⚠️ __Note:__ there are some municipalities with dominant cluster as _null_ because I didn't find match between both databases by comparing the name of the municipality. 

In [107]:
# source['dominant_cluster']#.unique()


print('\nmunicipalities', len(source), '\n')
print(81+313+645)
# source[source['dominant_cluster'] == "First"]

xxx = source.groupby(['dominant_cluster']).agg(['count'])[['your_municipality_resides']]
xxx


# source.groupby(['dominant_cluster'])#.apply(lambda x: 100 * x / float(x.sum()))


municipalities 1039 

1039


,your_municipality_resides
,count
dominant_cluster,
lower_class,718
middle_class,270
upper_class,51


In [108]:
xxx / (1039)

,your_municipality_resides
,count
dominant_cluster,
lower_class,0.691049
middle_class,0.259865
upper_class,0.049086


The cluster mappings by Municipality shows a clear majority to the third cluster and a minority to the first cluster. From the 1,039 municipalities in the database, 645 municipalities belong to the third cluster (62.1%), 313 municipalities belong to the second cluster (30.1%), and 81 municipalities to the first cluster (7.8%). Also, some municipalities have null value as dominant cluster, which could be explained because of lack of information or errors to map the clusters between databases.

In [91]:
# create excel writer
writer = pd.ExcelWriter(root + 'Class_cluster.xlsx')

# write dataframe to excel sheet named 'marks'
cluster_dpto.to_excel(writer, 'Department')
cluster_mpio.to_excel(writer, 'Municipality')

# save the excel file
writer.save()
print('DataFrame is written successfully to Excel Sheet.')

DataFrame is written successfully to Excel Sheet.
